In [1]:
import tensorflow as tf

In [2]:
# define the image size and batch size
img_size = (40, 40)
batch_size = 32

# Path to your dataset folder
data_dir = "C:/Users/ASUS/Desktop/newest dataset mixed"

In [3]:
# use image_dataset_from_directory to load the data and split it into training and testing sets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=img_size,
    batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=img_size,
    batch_size=batch_size)

Found 22574 files belonging to 8 classes.
Using 18060 files for training.
Found 22574 files belonging to 8 classes.
Using 4514 files for validation.


In [4]:
# inspect the class names and number of classes
class_names = train_ds.class_names
num_classes = len(class_names)
print('Class names:', class_names)
print('Number of classes:', num_classes)

Class names: ['0', '1', '2', '3', '4', '5', '6', '7']
Number of classes: 8


In [5]:
# convert images to grayscale
def grayscale(image, label):
    image = tf.image.rgb_to_grayscale(image)
    return image, label

train_ds = train_ds.map(grayscale)
test_ds = test_ds.map(grayscale)

# normalize pixel values to be between 0 and 1
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
test_ds = test_ds.map(lambda x, y: (x / 255.0, y))

In [6]:
# create the neural network
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(40, 40, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# The neural network architecture consists of sequential layers, where the output of one layer is passed as input to the next layer.

# The first layer is a convolutional layer with 16 filters of size 3x3, followed by a ReLU activation function. 
# The input shape of the layer is (40, 40, 1), which means that the layer expects input images of size 40x40 with one color channel (grayscale).

# The second layer is a max pooling layer that reduces the spatial dimensions of the output from the previous layer by taking the maximum value in each 2x2 window.

# The third layer is another convolutional layer with 32 filters of size 3x3, followed by a ReLU activation function.

# The fourth layer is another max pooling layer.

# The fifth layer is a flatten layer that flattens the output from the previous layer into a 1D vector.

# The sixth layer is a dense layer with 64 units and a ReLU activation function.

# The seventh and final layer is a dense layer with num_classes units and a softmax activation function, which produces the predicted class probabilities for the input image.

In [16]:
# # For visualizing the model

# from keras.utils.vis_utils import plot_model
# plot_model(model, 
#            to_file='model.png',
#            show_shapes=True,
#            show_dtype=False,
#            show_layer_names=True,
#            rankdir='TB',
#            expand_nested=False,
#            dpi=300,
#            layer_range=None,
#            show_layer_activations=True
#           )

In [8]:
# from ann_visualizer.visualize import ann_viz;

# ann_viz(model, title="My first neural network")

In [9]:
# compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
# train the model
model.fit(train_ds, validation_data=test_ds, epochs=10)

Epoch 1/10


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


565/565 [==============================] - 17s 28ms/step - loss: 1.1001 - accuracy: 0.6290 - val_loss: 0.5732 - val_accuracy: 0.8327
Epoch 2/10
565/565 [==============================] - 10s 17ms/step - loss: 0.4503 - accuracy: 0.8661 - val_loss: 0.3638 - val_accuracy: 0.8919
Epoch 3/10
565/565 [==============================] - 11s 19ms/step - loss: 0.2952 - accuracy: 0.9148 - val_loss: 0.2782 - val_accuracy: 0.9176
Epoch 4/10
565/565 [==============================] - 11s 19ms/step - loss: 0.2167 - accuracy: 0.9381 - val_loss: 0.2155 - val_accuracy: 0.9395
Epoch 5/10
565/565 [==============================] - 11s 19ms/step - loss: 0.1650 - accuracy: 0.9524 - val_loss: 0.1977 - val_accuracy: 0.9444
Epoch 6/10
565/565 [==============================] - 11s 19ms/step - loss: 0.1233 - accuracy: 0.9652 - val_loss: 0.1971 - val_accuracy: 0.9451
Epoch 7/10
565/565 [==============================] - 10s 18ms/step - loss: 0.1051 - accuracy: 0.9685 - val_loss: 0.2277 - val_accuracy: 0.9402
Epo

In [17]:
loss, accuracy = model.evaluate(test_ds, verbose=0)

print(f'Testing accuracy: {(accuracy*100):.2f}%')

Testing accuracy: 95.88%


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 38, 38, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 19, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 64)                1

In [19]:
model.save('MP_new_dataset.h5')